# Set up workspace

In [5]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from scipy.stats import ttest_ind
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
#from eofs.xarray import Eof 

In [6]:
import cftime
import scipy

In [7]:
import sys
sys.path.append('/glade/u/home/czarakas/coupled_PPE/code/utils')

import make_multisimulation_dataset

In [8]:
from load_ensembles import *
import quick_map

In [9]:
path_in='/glade/work/czarakas/coupled_PPE/data/data_for_figures/timeseries/'
path_out='/glade/work/czarakas/coupled_PPE/data/data_for_figures/annual_means/'

# Load data

### Define variables to load

In [10]:
var='TSKIN'#'calculated_PREC_FROM_ATM'#'TSKIN'
domain='lnd'
ensemble='offline'
season='Annual'

In [11]:
end_spinup=40

### Load full ensemble data

In [12]:
var_array_withspinup=xr.open_dataset(path_in+var+'_timeseries_'+ensemble+'.nc')

In [13]:
var_array=var_array_withspinup[var][end_spinup*12:,:,:,:]

### Define function for calculating annual averages based on days per year

In [14]:
dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '365_day': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'standard': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'all_leap': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '366_day': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]}

def get_dpm(time, calendar='standard'):
    """
    return a array of days per month corresponding to the months provided in `months`
    """
    month_length = np.zeros(len(time), dtype=np.int)

    cal_days = dpm[calendar]

    for i, (month, year) in enumerate(zip(time.month, time.year)):
        month_length[i] = cal_days[month]
    return month_length

In [15]:
if ensemble=='offline':
    ensemble_path='offline_simulations'
    refcase_id='OFFL0000_PI_v02'
elif ensemble=='coupled':
    ensemble_path='coupled_simulations'
    refcase_id='COUP0000_PI_SOM'
if domain=='atm':
    domain_suffix='.cam.h0.'
elif domain=='lnd':
    domain_suffix='.clm2.h0.'
    
basecase_lnd=xr.open_dataset('/glade/campaign/cgd/tss/czarakas/CoupledPPE/'+ensemble_path+'/'+
                                 refcase_id+'/'+domain+'/proc/tseries/'+refcase_id+domain_suffix+'timeseries.'+var+'.nc')

In [16]:
ds=basecase_lnd
month_length = xr.DataArray(get_dpm(ds.time.to_index(),
                                      calendar='noleap'),
                              coords=[ds.time], name='month_length')

numyears=np.size(ds.time.groupby('time.year').mean(dim='time').year)

In [17]:
# Calculate the weights by grouping by 'time.season'.
# Conversion to float type ('astype(float)') only necessary for Python 2.x
weights = month_length.groupby('time.year') / month_length.astype(float).groupby('time.year').sum()

# Test that the sum of the weights for each season is 1.0
#weights.groupby('time.month').sum().values
np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(numyears))

# Calculate the weighted average
ds_weighted = (ds[var] * weights).groupby('time.year').sum(dim='time')

In [18]:
def calculate_annual_timeseries(da, season='Annual',
                               end_spinup=0):
    month_length = xr.DataArray(get_dpm(ds.time.to_index(),
                                      calendar='noleap'),
                              coords=[ds.time], name='month_length')
    
    numyears=np.size(ds.time.groupby('time.year').mean(dim='time'))
    
    if season=='Annual':
        weights = month_length.groupby('time.year') / month_length.astype(float).groupby('time.year').sum()
        
        # Test that the sum of the weights for each year is 1.0
        np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(numyears))
    
        # Calculate the weighted average
        da_weighted = (da * weights).groupby('time.year').sum(dim='time')
        
        this_ensemble_tseries=da_weighted[end_spinup:,:,:]
    else:
        months_filtered = month_length.where(month_length['time.season']==season)
        weights = months_filtered.groupby('time.year') / months_filtered.astype(float).groupby('time.year').sum()
        
        # Test that the sum of the weights for each season is 1.0
        np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(numyears))
        
        da_weighted = (da * weights).groupby('time.year').sum(dim='time')
        this_ensemble_tseries=da_weighted[end_spinup:,:,:]
        
    return this_ensemble_tseries
    

# Do calculations

In [19]:
var_ref_annual=calculate_annual_timeseries(basecase_lnd[var][end_spinup*12:,:,:],season=season)
var_ref_mean=var_ref_annual.mean(dim='year')
ds_ref=var_ref_mean.to_dataset(name=var)

In [20]:
var_array_annual=calculate_annual_timeseries(var_array,season=season)
var_array_mean=var_array_annual.mean(dim='year')
ds_mean=var_array_mean.to_dataset(name=var)

In [21]:
var_array_delta=var_array_mean-var_ref_mean
ds_delta=var_array_delta.to_dataset(name=var)

# Save summary stats

In [ ]:
fname=var+'_'+season+'_mean_'+ensemble+'.nc'
ds_mean.to_netcdf(path_out+fname)

fname='ref_'+var+'_'+season+'_mean_'+ensemble+'.nc'
ds_ref.to_netcdf(path_out+fname)

fname='delta_'+var+'_'+season+'_mean_'+ensemble+'.nc'
ds_delta.to_netcdf(path_out+fname)

# Calculate where changes are significant

In [22]:
ds_grid = basecase_lnd

var_array_ttest = make_multisimulation_dataset.make_empty_dataarray(ds_grid=ds_grid, var=var, keys=keys)
var_array_pval = make_multisimulation_dataset.make_empty_dataarray(ds_grid=ds_grid, var=var, keys=keys)

/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [23]:
for i, key in enumerate(var_array.ensemble_key):
    ds_annual=var_array_annual[:,:,:,i]
    [tstat, pval] = scipy.stats.ttest_ind(var_ref_annual, ds_annual, equal_var=False)#, alternative='two-sided')
    var_array_ttest[:,:,i]=tstat
    var_array_pval[:,:,i]=pval

/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/glade/u/home/czarakas/.conda/envs/EOF_env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [24]:
ds_pval=var_array_pval.to_dataset(name='pval')

fname='pval_'+var+'_'+season+'_mean_'+ensemble+'.nc'
ds_pval.to_netcdf(path_out+fname)